<a href="https://colab.research.google.com/github/kartikrupal/Hate-Speech/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import spacy
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE

Loading Dataset

In [2]:
# Install this in your environment if not already done
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

# File inside the dataset
file_path = "labeled_data.csv"

# Load the dataset
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mrmorj/hate-speech-and-offensive-language-dataset",
    file_path
)

# Display the first few rows
print(df.head())


<ipython-input-2-f173612dff90>:11: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 1.01M/1.01M [00:00<00:00, 71.9MB/s]

Extracting zip of labeled_data.csv...
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [3]:
df.head(15)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [4]:
df.columns

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')

Dataset Cleaning

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [6]:
df.isnull().sum()

,0
Unnamed: 0,0
count,0
hate_speech,0
offensive_language,0
neither,0
class,0
tweet,0


In [7]:
df.drop(columns=['count','hate_speech','offensive_language','neither','Unnamed: 0'],inplace=True)

In [8]:
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


removing all the special characters

In [9]:
df['New_tweet']=df['tweet'].str.replace(r'[^a-zA-Z]',' ',regex=True)

In [10]:
df

,class,tweet,New_tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldn t...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats cold tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C G Anderson viva based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you...
...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,you s a muthaf in lie LifeAsKing ...
24779,2,"you've gone and broke the wrong heart baby, an...",you ve gone and broke the wrong heart baby an...
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [11]:
df['New_tweet'].iloc[5]

'                    T Madison x  The shit just blows me  claim you so faithful and down for somebody but still fucking with hoes                              '

In [12]:
df['New_tweet']=df['New_tweet'].str.replace(r'[\s]+',' ',regex=True)

In [13]:
df

,class,tweet,New_tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldn t comp...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats cold tyga dwn bad for cuffi...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbaby life You ever ...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C G Anderson viva based she look like a tr...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you hear about me ...
...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,you s a muthaf in lie LifeAsKing Pearls corey ...
24779,2,"you've gone and broke the wrong heart baby, an...",you ve gone and broke the wrong heart baby and...
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like I aint fu...
24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [14]:
df['New_tweet'].iloc[5]

' T Madison x The shit just blows me claim you so faithful and down for somebody but still fucking with hoes '

In [15]:
df.drop(columns=['tweet'],inplace=True)

In [16]:
df

,class,New_tweet
0,2,RT mayasolovely As a woman you shouldn t comp...
1,1,RT mleew boy dats cold tyga dwn bad for cuffi...
2,1,RT UrKindOfBrand Dawg RT sbaby life You ever ...
3,1,RT C G Anderson viva based she look like a tr...
4,1,RT ShenikaRoberts The shit you hear about me ...
...,...,...
24778,1,you s a muthaf in lie LifeAsKing Pearls corey ...
24779,2,you ve gone and broke the wrong heart baby and...
24780,1,young buck wanna eat dat nigguh like I aint fu...
24781,1,youu got wild bitches tellin you lies


***NLP PROCESSING***

In [17]:
import spacy

In [18]:
nlp=spacy.load('en_core_web_sm')


In [19]:
def lemmatization(text):
  doc=nlp(text)
  lemmatized = [word.lemma_ for word in doc]
  return " ".join(lemmatized)


In [20]:
text="kartik is in 3rd year"
lemmatization(text)

'kartik be in 3rd year'

In [21]:
df['processed_text']=df['New_tweet'].apply(lemmatization)

In [22]:
df.head(10)

,class,New_tweet,processed_text
0,2,RT mayasolovely As a woman you shouldn t comp...,RT mayasolovely as a woman you shouldn t com...
1,1,RT mleew boy dats cold tyga dwn bad for cuffi...,RT mleew boy dat cold tyga dwn bad for cuffi...
2,1,RT UrKindOfBrand Dawg RT sbaby life You ever ...,RT UrKindOfBrand Dawg RT sbaby life you ever...
3,1,RT C G Anderson viva based she look like a tr...,RT C G Anderson viva base she look like a tr...
4,1,RT ShenikaRoberts The shit you hear about me ...,RT ShenikaRoberts the shit you hear about I ...
5,1,T Madison x The shit just blows me claim you ...,T Madison x the shit just blow I claim you s...
6,1,BrighterDays I can not just sit up and HATE o...,BrighterDays I can not just sit up and HATE ...
7,1,selfiequeenbri cause I m tired of you big bit...,selfiequeenbri cause I m tired of you big bi...
8,1,amp you might not get ya bitch back amp thats...,amp you might not get ya bitch back amp that...
9,1,rhythmixx hobbies include fighting Mariam bitch,rhythmixx hobby include fight Mariam bitch


In [23]:
def stop_removal(text):
  doc=nlp(text)
  filtered_tokens = [token.text for token in doc if not token.is_stop]
  return " ".join(filtered_tokens)

In [24]:
df['processed_text_2']=df['processed_text'].apply(stop_removal)

In [25]:
df

,class,New_tweet,processed_text,processed_text_2
0,2,RT mayasolovely As a woman you shouldn t comp...,RT mayasolovely as a woman you shouldn t com...,RT mayasolovely woman shouldn t complain cl...
1,1,RT mleew boy dats cold tyga dwn bad for cuffi...,RT mleew boy dat cold tyga dwn bad for cuffi...,RT mleew boy dat cold tyga dwn bad cuffin d...
2,1,RT UrKindOfBrand Dawg RT sbaby life You ever ...,RT UrKindOfBrand Dawg RT sbaby life you ever...,RT UrKindOfBrand Dawg RT sbaby life fuck bi...
3,1,RT C G Anderson viva based she look like a tr...,RT C G Anderson viva base she look like a tr...,RT C G Anderson viva base look like tranny
4,1,RT ShenikaRoberts The shit you hear about me ...,RT ShenikaRoberts the shit you hear about I ...,RT ShenikaRoberts shit hear true faker bitc...
...,...,...,...,...
24778,1,you s a muthaf in lie LifeAsKing Pearls corey ...,you s a muthaf in lie lifeaske Pearls corey em...,s muthaf lie lifeaske Pearls corey emanuel rig...
24779,2,you ve gone and broke the wrong heart baby and...,you ve go and break the wrong heart baby and d...,ve break wrong heart baby drive redneck crazy
24780,1,young buck wanna eat dat nigguh like I aint fu...,young buck wanna eat dat nigguh like I be not ...,young buck wanna eat dat nigguh like fuckin dis
24781,1,youu got wild bitches tellin you lies,youu get wild bitch tellin you lie,youu wild bitch tellin lie


In [26]:
df.drop(columns=['New_tweet','processed_text'],inplace=True)

In [27]:
df

,class,processed_text_2
0,2,RT mayasolovely woman shouldn t complain cl...
1,1,RT mleew boy dat cold tyga dwn bad cuffin d...
2,1,RT UrKindOfBrand Dawg RT sbaby life fuck bi...
3,1,RT C G Anderson viva base look like tranny
4,1,RT ShenikaRoberts shit hear true faker bitc...
...,...,...
24778,1,s muthaf lie lifeaske Pearls corey emanuel rig...
24779,2,ve break wrong heart baby drive redneck crazy
24780,1,young buck wanna eat dat nigguh like fuckin dis
24781,1,youu wild bitch tellin lie


In [28]:
df = df.rename(columns={'processed_text_2': 'content'})
df

,class,content
0,2,RT mayasolovely woman shouldn t complain cl...
1,1,RT mleew boy dat cold tyga dwn bad cuffin d...
2,1,RT UrKindOfBrand Dawg RT sbaby life fuck bi...
3,1,RT C G Anderson viva base look like tranny
4,1,RT ShenikaRoberts shit hear true faker bitc...
...,...,...
24778,1,s muthaf lie lifeaske Pearls corey emanuel rig...
24779,2,ve break wrong heart baby drive redneck crazy
24780,1,young buck wanna eat dat nigguh like fuckin dis
24781,1,youu wild bitch tellin lie


In [29]:
df['content'].iloc[67]

'   Allyhaaaaa Lemmie eat Oreo amp dish oreo lol'

**ONE HOT ENCODING**

In [30]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
vocab_size=10000
one_hot_representation=[ one_hot(words,vocab_size) for words in df['content']]

In [32]:
for i in range(6):
  print(df['content'].iloc[i])

   RT mayasolovely woman shouldn t complain clean house amp man trash
   RT mleew boy dat cold tyga dwn bad cuffin dat hoe st place
   RT UrKindOfBrand Dawg RT sbaby life fuck bitch start cry confuse shit
   RT C G Anderson viva base look like tranny
   RT ShenikaRoberts shit hear true faker bitch tell ya
   T Madison x shit blow claim faithful somebody fuck hoe


In [33]:
for i in range(6):
  print(one_hot_representation[i])


[2101, 7725, 9225, 8714, 4998, 8624, 7333, 9359, 9267, 2107, 8958]
[2101, 5807, 6552, 5078, 5313, 9421, 6301, 4944, 4887, 5078, 5272, 732, 5311]
[2101, 3202, 7304, 2101, 421, 6556, 9028, 5922, 8656, 807, 3338, 7945]
[2101, 8303, 7657, 6798, 6810, 8017, 3836, 4614, 98]
[2101, 9979, 7945, 3053, 2096, 1019, 5922, 8330, 2066]
[4998, 5251, 9969, 7945, 8151, 7558, 518, 4257, 9028, 5272]


In [34]:
sentence_length= 20
embedded_tweet=pad_sequences(one_hot_representation,padding='pre',maxlen=sentence_length)

In [35]:
for i in range(6):
  print(embedded_tweet[i])


[   0    0    0    0    0    0    0    0    0 2101 7725 9225 8714 4998
 8624 7333 9359 9267 2107 8958]
[   0    0    0    0    0    0    0 2101 5807 6552 5078 5313 9421 6301
 4944 4887 5078 5272  732 5311]
[   0    0    0    0    0    0    0    0 2101 3202 7304 2101  421 6556
 9028 5922 8656  807 3338 7945]
[   0    0    0    0    0    0    0    0    0    0    0 2101 8303 7657
 6798 6810 8017 3836 4614   98]
[   0    0    0    0    0    0    0    0    0    0    0 2101 9979 7945
 3053 2096 1019 5922 8330 2066]
[   0    0    0    0    0    0    0    0    0    0 4998 5251 9969 7945
 8151 7558  518 4257 9028 5272]


In [36]:
X=np.array(embedded_tweet)
y=np.array(df['class'])
df['class'].value_counts()

,count
class,
1,19190
2,4163
0,1430


Fixing the inconsistent data oversampling

In [37]:
smote=SMOTE(sampling_strategy='minority')
X,y=smote.fit_resample(X,y)


***Model Building***

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)